In [2]:
if (!require(rvest)) install.packages('rvest')
library(rvest)

if (!require(tidyverse)) install.packages('tidyverse')
library(tidyverse)

base_url <- 'http://gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL'

In [3]:
getLinks <- function(termGbn){
    base_url <- 'http://gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL'
    sample_url <- paste0(base_url, '&termGbn=', termGbn)
    dates <-     
        read_html(sample_url) %>%
        html_nodes('div[class=fr]') %>%
        html_nodes('select') %>%
        html_nodes('option') %>%
        html_attr('value') %>%
        str_subset(pattern = '^[0-9]{6}$')
    
    hitYear <-
        dates %>%
        substr(1,4) # 연도 ex) 2022 => 2022년

    targetTime <-
        dates %>%
        substr(5,6) # 몇 주차 또는 몇 월차인지 ex) 14 => 14주차
    
    completed_url <- paste0(base_url, '&targetTime=', targetTime, '&hitYear=', hitYear, '&termGbn=', termGbn)
    return(completed_url)
}

In [4]:
getPage <- function(target_url){
    year_week <- 
        target_url %>%
        strsplit('=|&') %>%
        unlist() %>%
        str_subset('^[0-9]{2,4}$') %>%
        rev() %>%
        paste0(collapse = '')

    html_chart <- read_html(target_url) %>%
        html_nodes('div[class=chart]')

    td <- html_chart %>% 
        html_nodes('td') %>%
        html_text() %>%
        str_split('\\n|\\||\\r|\\t') %>%
        unlist() %>%
        str_subset('^$', negate = TRUE)

    change_grp <- html_chart %>%
        html_nodes('td[class=change]') %>%
        html_nodes('span') %>%
        html_attr('class')

    index_PLAY <- str_which(td, 'PLAY')[1]

    chart_piece <- td %>% 
        matrix(ncol = index_PLAY, byrow = TRUE) %>%
        as_tibble()

    # Note : ifelse always returns an object of the same length as the condition. so use if/else for this case
    chart <- data.frame(year_week = year_week,
                        ranking = chart_piece$V1,
                        change_grp = change_grp,
                        change_val = chart_piece$V2,
                        title = chart_piece$V3,
                        artist = chart_piece$V4,
                        gaon_index = if (ncol(chart_piece) == 12) chart_piece$V6 else NA,  
                        production = if(ncol(chart_piece) == 12) chart_piece$V7 else chart_piece$V6,
                        distribution = if(ncol(chart_piece) == 12) chart_piece$V8 else chart_piece$V7)

    return(chart)
}

In [8]:
weekLinks <- getLinks('week')

In [9]:
getPage(weekLinks[422])

Warning message:
"The `x` argument of `as_tibble.matrix()` must have unique column names if `.name_repair` is omitted as of tibble 2.0.0.
Using compatibility `.name_repair`.
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was generated."
Warning message:
"Unknown or uninitialised column: `V6`."
Warning message:
"Unknown or uninitialised column: `V7`."


ERROR: Error in data.frame(year_week = year_week, ranking = chart_piece$V1, change_grp = change_grp, : arguments imply differing number of rows: 1, 220, 100, 0


In [ ]:
start <- Sys.time()
weekLinks <- getLinks('week')
all_pages <- vector('list', length(weekLinks))
for (i in 1:length(all_pages)) all_pages[[i]] <- getPage(weekLinks[i])
end <- Sys.time()

In [14]:
c('123,413,412', '31,123', '123', NA) %>% str_replace_all('[^0-9]', '')

[1] "123413412" "31123"     "123"       NA

In [ ]:
final <- do.call('rbind', all_pages)
write.csv(final, 'gaon_week.txt', row.names = FALSE)
write.table(final, 'gaon_week.txt', row.names = FALSE)


In [15]:
NA %>% as.numeric()

[1] NA